### Limit the size of the original POS songs to get a reduced dataset with fewer padding values

In [1]:
# start by feeding your Pickle functions to call and save pickle variables later on

import pickle
def writePickle( Variable, fname):
    filename = fname +".pkl"
    f = open("pickle_vars/"+filename, 'wb')
    pickle.dump(Variable, f)
    f.close()
def readPickle(fname):
    filename = "pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj
def readPicklefromPast(fname):
    filename = "../pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
# load the previously formed POS datasets, prepared for training
train_POS_input_data = readPickle("cnn_data_inputs/train_POS_input_data")
train_POS_labels = readPickle("cnn_data_inputs/train_POS_labels")
dev_POS_input_data = readPickle("cnn_data_inputs/dev_POS_input_data")
dev_POS_labels = readPickle("cnn_data_inputs/dev_POS_labels")
test_POS_input_data = readPickle("cnn_data_inputs/test_POS_input_data")
test_POS_labels = readPickle("cnn_data_inputs/test_POS_labels")

In [ ]:
import random

def POS_refiner(input_data):
    new_input_data = list()
    for song in input_data:
        reduced_song = []
        song_line_ruler = list()
        for line in song:
            if len(list(filter(lambda a: a != 0, line))) != 0:
                new_line = list(filter(lambda a: a != 0, line))
                song_line_ruler.append((song.index(line),len(new_line)))
            else:
                song_line_ruler.append((song.index(line),0))
        song_line_ruler = list(set(song_line_ruler)) # remove duplicate lines
        song_line_ruler.sort(key=lambda x: x[1], reverse=True) # sort by the number of POS tags present
        song_line_ruler = song_line_ruler[0:20] # take the most frequent 20
        song_line_ruler.sort(key=lambda x: x[0]) # sort them back in shape wrt their original places in the song

        for item in song_line_ruler:
            reduced_song.append(song[item[0]])
        refined_song = list()
        for line in reduced_song:
            if set(line) <= {0}: # if the line is all blanks, replace the line with ten random integers up to 18
                refined_song.append([random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18)])
            else:
                refined_line = list(filter(lambda a: a != 0, line))
                if len(refined_line) > 10: # if there are more than ten elements, take the last 10 of them
                    refined_line = refined_line[-10:]
                else: # if there are only less than 10 elements
                    diff = 10 - len(refined_line)
                    for _ in range(diff): # randomly fill POS tags in 
                        refined_line.insert(random.randrange(0,len(refined_line),1),random.randint(0,18))
                refined_song.append(refined_line)
        if len(refined_song) < 20:
            diff = 20 - len(refined_song)
            for _ in range(diff):
                blank_line = [random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18),random.randint(0,18)]
                refined_song.append(blank_line)
            
        new_input_data.append(refined_song)
    return new_input_data
            

In [ ]:
train_POS_reduced_input_data = POS_refiner(train_POS_input_data)
dev_POS_reduced_input_data = POS_refiner(dev_POS_input_data)
test_POS_reduced_input_data = POS_refiner(test_POS_input_data)


In [ ]:
# convert all of them to numpy arrays, so that they can be used in keras
import numpy as np

train_POS_input_data = np.array(train_POS_reduced_input_data)
train_POS_labels = np.array(train_POS_labels)
dev_POS_input_data = np.array(dev_POS_reduced_input_data)
dev_POS_labels = np.array(dev_POS_labels)
test_POS_input_data = np.array(test_POS_reduced_input_data)
test_POS_labels = np.array(test_POS_labels)

In [ ]:
# for all the input data, we have to normalize the data points to an interval between 0 and 1, 
# and convert everything to floating numbers
train_POS_input_data = train_POS_input_data.astype('float32') / np.amax(train_POS_input_data)
dev_POS_input_data = dev_POS_input_data.astype('float32') / np.amax(dev_POS_input_data)
test_POS_input_data = test_POS_input_data.astype('float32') / np.amax(test_POS_input_data)

In [ ]:
# an example
train_POS_input_data[0]

In [ ]:
# reshape input

X_train_POS = train_POS_input_data.reshape(len(train_POS_input_data),20,10,1)
X_dev_POS = dev_POS_input_data.reshape(len(dev_POS_input_data),20,10,1)
X_test_POS = test_POS_input_data.reshape(len(test_POS_input_data),20,10,1)

# an example
print(X_train_POS.shape)

In [ ]:
# then we need to turn the labels into categorical values

from keras.utils import to_categorical

y_train_POS = to_categorical(train_POS_labels)
y_dev_POS = to_categorical(dev_POS_labels)
y_test_POS = to_categorical(test_POS_labels)

In [ ]:
# record all of them to pickle files
writePickle(X_train_POS,"cnn_data_inputs/POS_Keras/X_train_POS_reduced")
writePickle(X_dev_POS,"cnn_data_inputs/POS_Keras/X_dev_POS_reduced")
writePickle(X_test_POS,"cnn_data_inputs/POS_Keras/X_test_POS_reduced")
writePickle(y_train_POS,"cnn_data_inputs/POS_Keras/y_train_POS_reduced")
writePickle(y_dev_POS,"cnn_data_inputs/POS_Keras/y_dev_POS_reduced")
writePickle(y_test_POS,"cnn_data_inputs/POS_Keras/y_test_POS_reduced")

### From the this moment on, we will use the Model scripts to construct our models, using the pickle input variables right above